# SPELL CHECKER NYU CDS
### Group: ssp570 man514
### Members:
### 1. Supriya Pai
### 2. Manish Nagdevani

References: Peter Norvig blog: https://norvig.com/spell-correct.html

In [190]:
import re
from collections import Counter

words() Function returns word tokens from the input text

In [191]:
def words(text): return re.findall('[a-z]+', text.lower())

Store frequency of words in dictionary WORDS as {word:frequency}

In [192]:
WORDS = Counter(words(open('/Users/Manish/Desktop/ids/Assignments/asn2/big.txt').read()))

P() Function returns the probability of a word occurence in the corpus represented by WORDS

In [193]:
def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

words_split() Function splits the word on colon as a delimeter

In [194]:
def words_split(text): return text.split(":")

Read the spell error file and maintain a lookup table as:
{incorrect word: set(correct words)}

In [195]:
lookup = {}
with open('/Users/Manish/Desktop/ids/Assignments/asn2/spell-errors.txt') as f:
    for line in f:
        dict = words_split(line)        
        for w in words(dict[1]):
            #print w
            a = set()
            if w in lookup:
                a = lookup[w]
            
            a.add(dict[0])
            lookup[w] = a

correction() Function predicts a corrected word for the input incorrect word

In [196]:
def correction(word):
    "Most probable spelling correction for word."
    candidate_pool = candidates(word)
    if word in lookup:
        a = lookup[word]
        return max(a, key=P)
        #candidate_pool = candidate_pool.union(a)
        
    return max(candidate_pool, key=P)

Helper Functions

In [197]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or {word})

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Read Input Test File

In [198]:
import pandas as pd
df = pd.read_csv('/Users/Manish/Desktop/ids/Assignments/asn2/test.csv')

Generate correct words for all the incorrect words

In [199]:
correct_words = []
for wrong_word in df.WRONG:
    correct_words.append(correction(wrong_word))

Create a CSV of output

In [201]:
correct_words_df = pd.DataFrame(correct_words, columns=['CORRECT'])
correct_words_df.to_csv(columns=['CORRECT'],path_or_buf='/Users/Manish/Desktop/submit.csv', index_label="ID")